## The purpose of the Capstone 1 project is to provide a list of recommendation to a gamer so that they may find new games that will interest them.

"SVDpp Collaborative game recommendation" will use SVD++ algorithm to provide a list of reommendation for people who are new to the website and therefore has no prior data on the purchase or playtime

Cleaned game review data is imported from the pickle file

In [62]:
import pickle
file_name='cleaned_review.p'
loaded_df=pickle.load(open(file_name,"rb"))
steamapp_dict=pickle.load(open("steam.p","rb"))

In [63]:
loaded_df.head()

,author,review,game_name
0,76561198831801803,dont know how to use soundtrack or comic :/ hi...,Pikuniku Soundtrack + Comic
1,76561198208927921,"Bad game, totally worthless.",Four-color Fantasy
4,76561198063789203,It's 2:54 AM and I'm crying over winning a dra...,Drag Star!
5,76561198343959533,I literally stayed up til 3 am because I *need...,Drag Star!
6,76561198079874042,"I've never seen Drag Race, so I was a little u...",Drag Star!


In [3]:
loaded_df=loaded_df.drop_duplicates()

SVD++ algorithm from "surprise" library requires rating system to be in place. However, because we do not have rating provided in the raw data, we will get the sentiment values from the reviews and assign rating based on that

In [5]:
from textblob import TextBlob

In [6]:
review_sentiment=list()
i=0
for review in loaded_df.review:
    sent = TextBlob(review)
    rating=round(2*(sent.sentiment.polarity+1))
    review_sentiment.append(rating)

In [7]:
set(review_sentiment)

{0, 1, 2, 3, 4}

In [8]:
loaded_df['review_sentiment']=review_sentiment

In [10]:
from surprise import *

In [11]:
# Surprise takes user, item, and rating as input
df=loaded_df[['author','game_name','review_sentiment']]

In [12]:
# Only take games that have more than 5 reviews into consideration 
min_games_review = 5
filter_games = df['game_name'].value_counts() > min_games_review
filter_games = filter_games[filter_games].index.tolist()

# Only take users that left more than 5 reviews into consideration 
min_user_review = 5
filter_users = df['author'].value_counts() > min_user_review
filter_users = filter_users[filter_users].index.tolist()

df_new = df[(df['game_name'].isin(filter_games)) & (df['author'].isin(filter_users))]
print('The original data frame shape:\t{}'.format(df.shape))
print('The new data frame shape:\t{}'.format(df_new.shape))

The original data frame shape:	(79666, 3)
The new data frame shape:	(5373, 3)


In [13]:
# Make the reader and data form
reader = Reader(rating_scale=(0, 4))
data = Dataset.load_from_df(df_new, reader)

In [31]:
from surprise.model_selection import cross_validate
svdmodel=SVDpp()
cross_validate(svdmodel,data,measures=['RMSE'],cv=5,verbose=False)

{'test_rmse': array([0.4804533 , 0.47385527, 0.48043041, 0.47922555, 0.44107349]),
 'fit_time': (1.7401411533355713,
  1.80519437789917,
  1.755890130996704,
  1.7584702968597412,
  1.7765941619873047),
 'test_time': (0.03427314758300781,
  0.03236865997314453,
  0.03297090530395508,
  0.031549692153930664,
  0.03276538848876953)}

In [30]:
from surprise.model_selection import cross_validate
slopemodel=SVD()
cross_validate(slopemodel,data,measures=['RMSE'],cv=5,verbose=False)

{'test_rmse': array([0.45400046, 0.4772274 , 0.46113459, 0.48484196, 0.49460584]),
 'fit_time': (0.4381277561187744,
  0.4310038089752197,
  0.4149935245513916,
  0.4223594665527344,
  0.41359686851501465),
 'test_time': (0.01152348518371582,
  0.01294708251953125,
  0.012199640274047852,
  0.012076377868652344,
  0.012157201766967773)}

In [16]:
from surprise.model_selection import train_test_split
trainset,testset=train_test_split(data,test_size=0.3)

In [17]:
from surprise import accuracy
model=svdmodel.fit(trainset)
predict1=model.test(testset)
accuracy.rmse(predict1)

RMSE: 0.4877


0.48772951744906634

In [21]:
import pandas as pd
predict_df=pd.DataFrame(predict1)
predict_df['err'] = abs(predict_df.est - predict_df.r_ui)
best_predictions = predict_df.sort_values(by='err')[:10]
worst_predictions = predict_df.sort_values(by='err')[-10:]

In [22]:
best_predictions

,uid,iid,r_ui,est,details,err
757,76561198049701098,Crisis on the Planet of the Apes,2.0,2.000244,{'was_impossible': False},0.000244
1484,76561198234516860,Wild Island Quest,2.0,2.000295,{'was_impossible': False},0.000295
1272,76561198277984478,forma.8,2.0,2.000886,{'was_impossible': False},0.000886
180,76561198017125492,Death Train VR,2.0,1.998784,{'was_impossible': False},0.001216
835,76561198039580049,Lords of the Realm,2.0,1.998673,{'was_impossible': False},0.001327
313,76561198822401357,oVRshot,2.0,1.998647,{'was_impossible': False},0.001353
850,76561198137285867,GunWorld,2.0,1.998488,{'was_impossible': False},0.001512
856,76561197990354493,Away From Earth: Moon,2.0,1.998306,{'was_impossible': False},0.001694
965,76561198057056008,qop 2,2.0,2.001781,{'was_impossible': False},0.001781
179,76561198110829497,Escape Game,2.0,1.996785,{'was_impossible': False},0.003215


In [23]:
worst_predictions

,uid,iid,r_ui,est,details,err
400,76561197981047937,Fantasy Mosaics 16: Six Colors in Wonderland,4.0,2.075178,{'was_impossible': False},1.924822
1298,76561198006260288,Kingdom Defense,0.0,1.972678,{'was_impossible': False},1.972678
1474,76561198125100728,NARUTO SHIPPUDEN: Ultimate Ninja STORM Revolut...,4.0,2.014077,{'was_impossible': False},1.985923
449,76561198019816374,Eruption,4.0,2.006317,{'was_impossible': False},1.993683
448,76561198124516461,Old Friend,4.0,1.992652,{'was_impossible': False},2.007348
215,76561198031614337,Jackal,4.0,1.952587,{'was_impossible': False},2.047413
1485,76561197993862065,The Labyrinth of Time,0.0,2.065881,{'was_impossible': False},2.065881
990,76561198054387451,Bad Caterpillar,0.0,2.116034,{'was_impossible': False},2.116034
279,76561198057389389,Knights Hunt,0.0,2.317486,{'was_impossible': False},2.317486
396,76561198057389389,Hentai Defense,0.0,2.485438,{'was_impossible': False},2.485438


In [24]:
test_id='76561198141485825'
pred=model.predict(uid=test_id,iid="Fallout 4")
score=pred.est

In [33]:
import requests
# Get all the games that the player played
def getownedgames(idnum):
    """Takes the id number of a steam user and returns the id num of games they own"""
    url='http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key=3A629E52AE594CB777253431189F2967&steamid={}&format=json'.format(idnum)
    r=requests.get(url)
    json_data=r.json()
    return json_data

In [35]:
def rawplayergamelist(_id):
    gamelist=getownedgames(_id)['response']['games']
    return gamelist

In [37]:
testperson=rawplayergamelist(test_id)

In [38]:
testids=list()
testplaytimes=list()
for i in range(len(testperson)):
    testids.append(testperson[i]['appid'])
    testplaytimes.append(testperson[i]['playtime_forever'])

In [39]:
import pandas as pd

In [40]:
test_df=pd.DataFrame(list(zip(testids,testplaytimes)),columns=['ids','playtime']).sort_values('playtime',ascending=False)
playedgame_name=list()
for id_ in test_df['ids']:
    name=steamapp_dict.get(id_)
    playedgame_name.append(name)
test_df['name']=playedgame_name

In [41]:
tests=[[test_id,iid,5] for iid in predict_df["iid"].unique().tolist()]
predictions=model.test(tests)
pred_df=pd.DataFrame(predictions)

In [42]:
pred_df.head(10)

,uid,iid,r_ui,est,details
0,76561198141485825,Crashed Lander,5,2.116602,{'was_impossible': False}
1,76561198141485825,Geoid,5,2.127626,{'was_impossible': False}
2,76561198141485825,Tales from the Void,5,2.250663,{'was_impossible': False}
3,76561198141485825,Hiveswap Friendsim - Volume Five,5,2.062748,{'was_impossible': False}
4,76561198141485825,Train Simulator: BR Standard Class 6 ‘Clan Cla...,5,2.127626,{'was_impossible': False}
5,76561198141485825,The Travels of Marco Polo,5,2.166173,{'was_impossible': False}
6,76561198141485825,Brain Booster,5,2.127626,{'was_impossible': False}
7,76561198141485825,Turbo Pug 3D,5,2.127626,{'was_impossible': False}
8,76561198141485825,Siegecraft Commander,5,2.109553,{'was_impossible': False}
9,76561198141485825,Shmups Skill Test,5,2.116047,{'was_impossible': False}


In [43]:
test_df=pd.DataFrame(list(zip(testids,testplaytimes)),columns=['ids','playtime']).sort_values('playtime',ascending=False)
playedgame_name=list()
for id_ in test_df['ids']:
    name=steamapp_dict.get(id_)
    playedgame_name.append(name)

In [44]:
removed=[name for name in pred_df['iid'].tolist() if name not in playedgame_name]

In [45]:
pred_df=pred_df.set_index(['iid'])

In [46]:
finaltop10recommendation=pred_df.loc[removed].sort_values(['est'],ascending=False)[:10]

In [47]:
pred_game=finaltop10recommendation.index[0]
pred_rate=finaltop10recommendation['est'][0]
print('Top item for user {0} is {1} with predicted rating {2}'.format(test_id,pred_game, pred_rate))

Top item for user 76561198141485825 is Hiveswap Friendsim - Volume Nine with predicted rating 2.4689415780091597


Improving the model

In [48]:
from surprise.model_selection import GridSearchCV
param_grid={'lr_all' : [0.0001,.001,.01,.1], 'reg_all':[0.025,.05,.1,.5,.8]}
gs=GridSearchCV(SVDpp,param_grid,measures=['rmse','mae'],cv=3)
gs.fit(data)
print(gs.best_params['rmse']['lr_all'],gs.best_params['rmse']['reg_all'])

0.01 0.05


In [49]:
gs.best_params

{'rmse': {'lr_all': 0.01, 'reg_all': 0.05},
 'mae': {'lr_all': 0.01, 'reg_all': 0.5}}

In [50]:
new_model=SVDpp(lr_all=gs.best_params['rmse']['lr_all'],
                reg_all=gs.best_params['rmse']['reg_all'])
cross_validate(new_model,data,verbose=True)

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.4697  0.4656  0.4996  0.4560  0.4509  0.4684  0.0170  
MAE (testset)     0.2973  0.2965  0.3232  0.2957  0.2993  0.3024  0.0105  
Fit time          1.74    1.86    1.84    1.81    1.99    1.85    0.08    
Test time         0.03    0.03    0.03    0.03    0.04    0.03    0.00    


{'test_rmse': array([0.46965078, 0.46563617, 0.49964164, 0.45602992, 0.45087019]),
 'test_mae': array([0.29725166, 0.2964755 , 0.32318074, 0.29569054, 0.29927107]),
 'fit_time': (1.7410690784454346,
  1.8570973873138428,
  1.836747169494629,
  1.8138766288757324,
  1.992530345916748),
 'test_time': (0.033310890197753906,
  0.0349125862121582,
  0.03372383117675781,
  0.033187150955200195,
  0.03516507148742676)}

In [51]:
from surprise.model_selection import train_test_split
trainset,testset=train_test_split(data,test_size=0.3)

In [52]:
from surprise import accuracy
model2=new_model.fit(trainset)
predict2=model2.test(testset)
accuracy.rmse(predict2)

RMSE: 0.4809


0.48088426960330505

In [53]:
import pandas as pd
predict_df2=pd.DataFrame(predict2)
predict_df2['err'] = abs(predict_df2.est - predict_df2.r_ui)
best_predictions2 = predict_df2.sort_values(by='err')[:10]
worst_predictions2 = predict_df2.sort_values(by='err')[-10:]

In [54]:
best_predictions2

,uid,iid,r_ui,est,details,err
1534,76561197984712587,Fear of Clowns,2.0,2.000060,{'was_impossible': False},0.000060
572,76561197987731882,Spikit,2.0,2.000182,{'was_impossible': False},0.000182
128,76561197987731882,古剑奇谭(GuJian),2.0,2.000182,{'was_impossible': False},0.000182
417,76561198311688857,PippiStory,2.0,2.000570,{'was_impossible': False},0.000570
31,76561198377738658,Nancy Drew: The White Wolf of Icicle Creek,2.0,2.000597,{'was_impossible': False},0.000597
42,76561198214989148,Heroes of Legionwood,2.0,2.000809,{'was_impossible': False},0.000809
1194,76561197970480620,Cthulhu Realms,2.0,2.001065,{'was_impossible': False},0.001065
278,76561197982273259,Drive By Hero,2.0,1.998802,{'was_impossible': False},0.001198
293,76561198053422627,The Cursed Revolver,2.0,1.998518,{'was_impossible': False},0.001482
739,76561198027180717,Santa's Big Adventures,2.0,1.998332,{'was_impossible': False},0.001668


In [55]:
worst_predictions2

,uid,iid,r_ui,est,details,err
1093,76561198005539606,Escape Room VR: Stories,4.0,2.178106,{'was_impossible': False},1.821894
620,76561198114829485,It's Spring Again,4.0,2.157438,{'was_impossible': False},1.842562
559,76561198008426364,Enter the Gungeon - Soundtrack,4.0,2.129944,{'was_impossible': False},1.870056
169,76561197969484311,Gettysburg: The Tide Turns,4.0,2.108709,{'was_impossible': False},1.891291
1573,76561198054387451,Bad Caterpillar,0.0,1.942442,{'was_impossible': False},1.942442
29,76561198117674830,GameGuru - Expansion Pack,4.0,2.050212,{'was_impossible': False},1.949788
848,76561197993862065,The Labyrinth of Time,0.0,2.007750,{'was_impossible': False},2.007750
496,76561198019816374,Eruption,4.0,1.934327,{'was_impossible': False},2.065673
1423,76561197981715728,RDS - The Official Drift Videogame,4.0,1.864074,{'was_impossible': False},2.135926
1230,76561198124516461,Old Friend,4.0,1.835607,{'was_impossible': False},2.164393


The best predictions got a little worse but the worst predictions got a little better.

In [56]:
tests=[[test_id,iid,5] for iid in predict_df2["iid"].unique().tolist()]
new_predictions=new_model.test(tests)
pred_df2=pd.DataFrame(new_predictions)

In [57]:
pred_df2=pred_df2.set_index(['iid'])
new_top10recommendation=pred_df2.loc[removed].sort_values(['est'],ascending=False)[:10]

/home/seo/environments/my_env/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  


In [58]:
pred_game2=new_top10recommendation.index[0]
pred_rate2=new_top10recommendation['est'][0]
print('Top item for user {0} is {1} with predicted rating {2}'.format(test_id,pred_game2, pred_rate2))

Top item for user 76561198141485825 is Hiveswap Friendsim - Volume Nine with predicted rating 2.4364607194956545


In [59]:
top10=pred_df2.sort_values(['est'],ascending=False)[:10]
recommended_games=list()
for name in top10.index.tolist():
    ids=list(steamapp_dict.keys())[list(steamapp_dict.values()).index(name)]
    recommended_games.append((name,"https://store.steampowered.com/app/"+str(ids)))

In [60]:
top10

,uid,r_ui,est,details
iid,,,,
The Haunted Graveyard,76561198141485825,5,2.455778,{'was_impossible': False}
Hiveswap Friendsim - Volume Nine,76561198141485825,5,2.436461,{'was_impossible': False}
NARUTO SHIPPUDEN: Ultimate Ninja STORM Revolution- DLC3 Summer Cloth Pack,76561198141485825,5,2.428880,{'was_impossible': False}
Dead End Junction,76561198141485825,5,2.414303,{'was_impossible': False}
Caliper,76561198141485825,5,2.413487,{'was_impossible': False}
3DMark Fire Strike benchmark,76561198141485825,5,2.408227,{'was_impossible': False}
Tomb Raider: Hitman Gun - Agency SPS 12,76561198141485825,5,2.408033,{'was_impossible': False}
Abandoned Backpack,76561198141485825,5,2.393985,{'was_impossible': False}
Lunapark VR,76561198141485825,5,2.390232,{'was_impossible': False}


In [61]:
print("Based on what other players are playing a lot, here are some of the games you may like.")
print(pd.DataFrame(recommended_games,columns=["Game Name","Store page"]))

Based on what other players are playing a lot, here are some of the games you may like.
                                           Game Name  \
0                              The Haunted Graveyard   
1                   Hiveswap Friendsim - Volume Nine   
2  NARUTO SHIPPUDEN: Ultimate Ninja STORM Revolut...   
3                                  Dead End Junction   
4                                            Caliper   
5                       3DMark Fire Strike benchmark   
6            Tomb Raider: Hitman Gun - Agency SPS 12   
7                                 Abandoned Backpack   
8                                        Lunapark VR   
9                                   Galactic Gallery   

                                  Store page  
0  https://store.steampowered.com/app/931930  
1  https://store.steampowered.com/app/913810  
2  https://store.steampowered.com/app/301892  
3  https://store.steampowered.com/app/518280  
4  https://store.steampowered.com/app/581840  
5  https://st